In [1]:
import tensorflow as tf
print(tf.__version__)

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


# Shape error

Issue produce

In [8]:
def debug_shape(data):
  a = data[:, 0:2]
  b = data[:, 1]
  s = a + b
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  X = tf.constant([
      [1.0, 5.5, 4.5],
      [3.4, 6.7, 9.5],
      [3.2, 1.4, 7.2],
      [4.7, 8.8, 7.9]
    ])
  print(sess.run(debug_shape(X)))

ValueError: Dimensions must be equal, but are 2 and 4 for 'add_1' (op: 'Add') with input shapes: [4,2], [4].

#### Debug using print statements and fakedata

In [11]:
def debug_shape(data):
  a = data[:, 0:2]
  print(a.get_shape())
  b = data[:, 1]
  print(b.get_shape())
  s = a + b
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fakedata = tf.constant([
      [1.0, 5.5, 4.5],
      [3.4, 6.7, 9.5],
      [3.2, 1.4, 7.2],
      [4.7, 8.8, 7.9]
    ])
  print(sess.run(debug_shape(fakedata)))

(4, 2)
(4,)


ValueError: Dimensions must be equal, but are 2 and 4 for 'add_4' (op: 'Add') with input shapes: [4,2], [4].

#### solve shape error

In [12]:
def debug_shape(data):
  a = data[:, 0:2]
  print(a.get_shape())
  b = data[:, 1:3]
  print(b.get_shape())
  s = a + b
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fakedata = tf.constant([
      [1.0, 5.5, 4.5],
      [3.4, 6.7, 9.5],
      [3.2, 1.4, 7.2],
      [4.7, 8.8, 7.9]
    ])
  print(sess.run(debug_shape(fakedata)))

(4, 2)
(4, 2)
[[11.92686  15.088075 10.765685 15.960889]
 [15.088075 19.090574 13.630115 20.171515]
 [10.765685 13.630115  9.752948 14.342941]
 [15.960889 20.171515 14.342941 21.474173]]


# tf.expand_dims and tf.slice

In [16]:
x = tf.constant([
  [3, 2],
  [6, 8],
  [1, 7]
])
print("x.shape", x.shape)
expanded = tf.expand_dims(x, 1)
print("expanded.shape", expanded.shape)
sliced = tf.slice(x, [1,0], [2,1])
print("sliced.shape", sliced.shape)

with tf.Session() as sess:
  print("expanded: ", expanded.eval())
  print("sliced: ", sliced.eval())

('x.shape', TensorShape([Dimension(3), Dimension(2)]))
('expanded.shape', TensorShape([Dimension(3), Dimension(1), Dimension(2)]))
('sliced.shape', TensorShape([Dimension(2), Dimension(1)]))
('expanded: ', array([[[3, 2]],

       [[6, 8]],

       [[1, 7]]], dtype=int32))
('sliced: ', array([[6],
       [1]], dtype=int32))


# Vector vs Scalar

In [17]:
def some_method(data):
  print(data.get_shape())
  a = data[:,0:2]
  print(a.get_shape())
  c = data[:,1:3]
  print(c.get_shape())
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([5.0, 3.0, 7.1])
  print(sess.run(some_method(fake_data)))

(3,)


ValueError: Index out of range using input dim 1; input has only 1 dims for 'strided_slice_12' (op: 'StridedSlice') with input shapes: [3], [2], [2], [2] and with computed input tensors: input[3] = <1 1>.

In [20]:
def some_method(data):
  print(data.get_shape())
  a = data[:,0:2]
  print(a.get_shape())
  c = data[:,1:3]
  print(c.get_shape())
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([5.0, 3.0, 7.1])
  fake_data = tf.expand_dims(fake_data, 0)
  print(sess.run(fake_data))
  print(sess.run(some_method(fake_data)))

[[5.  3.  7.1]]
(1, 3)
(1, 2)
(1, 2)
[[12.884487]]


# Type error

In [24]:
def sum_types(a, b):
  s =  (a+b)
  return s

with tf.Session() as sess:
  X = tf.constant([3, 4, 5])
  Y = tf.constant([6.7, 7.7, 8.0])
  print(sess.run(sum_types(X, Y)))

ValueError: Tensor conversion requested dtype int32 for Tensor with dtype float32: 'Tensor("Const_20:0", shape=(3,), dtype=float32)'

In [26]:
def sum_types(a, b):
  a = tf.cast(a, tf.float32)
  s =  (a+b)
  return s

with tf.Session() as sess:
  X = tf.constant([3, 4, 5])
  Y = tf.constant([6.7, 7.7, 8.0])
  print(sess.run(sum_types(X, Y)))

[ 9.7 11.7 13. ]


# tf.Print()
Create a python script named debugger.py with the contents shown below.



In [27]:

%%writefile debugger.py
import tensorflow as tf

def some_method(a, b):
  b = tf.cast(b, tf.float32)
  s = (a / b)
  print_ab = tf.Print(s, [a, b])
  s = tf.where(tf.is_nan(s), print_ab, s)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_a = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ])
  fake_b = tf.constant([
      [2, 0, 5],
      [2, 8, 7]
    ])
  
  print(sess.run(some_method(fake_a, fake_b)))

Writing debugger.py


In [28]:
!ls -ltr

total 128
-rw-r--r-- 1 root root    20 May 14 13:33 README.md
-rw-r--r-- 1 root root    72 May 14 13:35 Untitled Notebook.ipynb
drwxr-xr-x 9 root root  4096 May 14 13:36 training-data-analyst
-rw-r--r-- 1 root root 62489 May 14 13:48 Tensorflow_Eager_Execution.ipynb
-rw-r--r-- 1 root root    72 May 14 13:52 Untitled Notebook1.ipynb
-rw-r--r-- 1 root root 41569 May 14 14:29 Debugging.ipynb
-rw-r--r-- 1 root root   434 May 14 14:33 debugger.py


#### Execute the python script

In [29]:
%%bash
python debugger.py

[[      inf       inf]
 [      inf 1.4336528]]


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2019-05-14 14:34:45.254575: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
[[5 3 7.1]...][[2 0 5]...]
